In [1]:
from smact import Element, ordered_elements, neutral_ratios
import itertools
import multiprocessing
from tqdm import tqdm
from functools import partial
from smact.screening import smact_filter
import os
from smact.data_loader import _get_data_rows_custom_ox_states
from smact.data_loader import lookup_element_oxidation_states_custom
from smact.data_loader import _get_data_rows
from smact.screening import pauling_test
from smact.utils.composition import comp_maker
from smact.utils.composition import formula_maker
from pprint import pprint

# from smact.utils.composition

In [2]:
max_atomic_num: int = 103
num_elements: int = 2
num_processes = None
max_stoich = 2
oxidation_states_set = "/Users/kinga/Documents/my_scripts/SMACT/commonality_oxstates_lists/low_commonality.txt"

# Starting with an oxidation states list

In [ ]:
_el_ox_states_custom = {}

for items in _get_data_rows_custom_ox_states_list(oxidation_states_set):
    _el_ox_states_custom[items[0]] = [
        int(oxidationState) for oxidationState in items[1:]
    ]

elements_pauling = [Element(element) for element in ordered_elements(1, max_atomic_num)]

strings_pauling = [
    element
    for element in ordered_elements(1, max_atomic_num)
    if element in _el_ox_states_custom.keys()
    and Element(element).pauling_eneg is not None
    and Element(element).pauling_eneg >= Element("Fr").pauling_eneg
]

compounds_pauling = list(itertools.combinations(elements_pauling, num_elements))

compounds_strings = list(itertools.combinations(strings_pauling, num_elements))

print(strings_pauling)
print(len(strings_pauling))
print(f"compounds_pauling: {compounds_pauling}")
print(f"type(compounds_pauling): {type(compounds_pauling)}")

ox_combos_str = [
    [_el_ox_states_custom.get(str1), _el_ox_states_custom.get(str2)]
    for str1, str2 in compounds_strings
]
print(f"ox_combos: {ox_combos_str}")

ox_combo = ox_combos_str[0]
print(f"ox_combo: {ox_combo}")

for ox_states in itertools.product(*ox_combo):
    print(f"ox_states: {ox_states}")
    # Test for charge balance
    cn_e, cn_r = neutral_ratios(ox_states, threshold=max_stoich)
    print(f"cn_e: {cn_e}, cn_r: {cn_r}")
    # Electronegativity test
    if cn_e:
        electroneg_OK = pauling_test(ox_states, electronegs)
        if electroneg_OK:
            for ratio in cn_r:
                compositions.append((symbols, ox_states, ratio))

# compounds_pauling_symbol = [(e1.symbol, e2.symbol) for e1, e2 in compounds_pauling]
# print(f"compounds_pauling: {compounds_pauling_symbol}")

# pool = multiprocessing.Pool(
#     processes=(multiprocessing.cpu_count() if num_processes is None else num_processes)
# )
# results = list(
#     tqdm(
#         pool.imap_unordered(
#             partial(
#                 smact_filter,
#                 threshold=max_stoich,
#                 oxidation_states_set=oxidation_states_set,
#             ),
#             compounds_pauling,
#         ),
#         total=len(compounds_pauling),
#     )
# )
# pool.close()
# pool.join()

# Looking inside smact_filter 

In [3]:
ox_states_custom = lookup_element_oxidation_states_custom(
    "all", oxidation_states_set, copy=False
)

elements_pauling = [
    Element(element)
    for element in ordered_elements(1, max_atomic_num)
    if element in ox_states_custom.keys()
    and Element(element).pauling_eneg is not None
    and Element(element).pauling_eneg >= Element("Fr").pauling_eneg
]

compounds_pauling = tuple(itertools.combinations(elements_pauling, num_elements))

print("#2. Generating all possible stoichiometric combinations...")

# for tuple in compounds_pauling:
#     results = smact_filter(
#         tuple, threshold=max_stoich, oxidation_states_set=oxidation_states_set
#     )

pool = multiprocessing.Pool(
    processes=(multiprocessing.cpu_count() if num_processes is None else num_processes)
)
results = list(
    tqdm(
        pool.imap_unordered(
            partial(
                smact_filter,
                threshold=max_stoich,
                oxidation_states_set=oxidation_states_set,
                as_tuple=True,
            ),
            compounds_pauling,
        ),
        total=len(compounds_pauling),
    )
)
pool.close()
pool.join()

print(results)

Returning the data for all elements.
#2. Generating all possible stoichiometric combinations...


  0%|          | 0/4186 [00:01<?, ?it/s]


MaybeEncodingError: Error sending result: '<generator object smact_filter at 0x13ad63340>'. Reason: 'TypeError("cannot pickle 'generator' object")'

In [16]:
print(results)

[[(('H', 'B'), (-1, 1), (1, 1)), (('H', 'B'), (-1, 2), (2, 1))], [(('H', 'N'), (1, -2), (2, 1)), (('H', 'N'), (1, -1), (1, 1))], [(('H', 'O'), (1, -2), (2, 1)), (('H', 'O'), (1, -1), (1, 1))], [(('H', 'F'), (1, -1), (1, 1))], [(('H', 'Na'), (-1, 1), (1, 1))], [(('H', 'Mg'), (-1, 2), (2, 1))], [], [(('H', 'Si'), (-1, 1), (1, 1)), (('H', 'Si'), (-1, 2), (2, 1))], [(('H', 'P'), (-1, 1), (1, 1)), (('H', 'P'), (-1, 2), (2, 1))], [(('H', 'S'), (1, -2), (2, 1)), (('H', 'S'), (1, -1), (1, 1))], [(('H', 'Cl'), (1, -1), (1, 1))], [(('H', 'K'), (-1, 1), (1, 1))], [(('H', 'Ca'), (-1, 1), (1, 1)), (('H', 'Ca'), (-1, 2), (2, 1))], [(('H', 'Sc'), (-1, 1), (1, 1)), (('H', 'Sc'), (-1, 2), (2, 1))], [(('H', 'Ti'), (-1, 1), (1, 1)), (('H', 'Ti'), (-1, 2), (2, 1))], [(('H', 'V'), (-1, 1), (1, 1)), (('H', 'V'), (-1, 2), (2, 1))], [(('H', 'Cr'), (-1, 1), (1, 1)), (('H', 'Cr'), (-1, 2), (2, 1))], [(('H', 'Mn'), (-1, 1), (1, 1)), (('H', 'Mn'), (-1, 2), (2, 1))], [(('H', 'Fe'), (-1, 1), (1, 1)), (('H', 'Fe'), 

In [ ]:
def smact_filter_new(
    els: tuple[Element] | list[Element],
    oxidation_states_set: str,
    threshold: int | None = 8,
    species_unique: bool = True,
) -> list[tuple[str, int, int]] | list[tuple[str, int]]:
    """Function that applies the charge neutrality and electronegativity
    tests in one go for simple application in external scripts that
    wish to apply the general 'smact test'.

    .. warning::
        For backwards compatibility in SMACT >=2.7, expllicitly set oxidation_states_set to 'smact14' if you wish to use the 2014 SMACT default oxidation states.
        In SMACT 3.0, the smact_filter function will be set to use a new default oxidation states set.

    Args:
    ----
        els (tuple/list): A list of smact.Element objects.
        threshold (int): Threshold for stoichiometry limit, default = 8.
        stoichs (list[int]): A selection of valid stoichiometric ratios for each site.
        species_unique (bool): Whether or not to consider elements in different oxidation states as unique in the results.
        oxidation_states_set (string): A string to choose which set of oxidation states should be chosen. Options are 'smact14', 'icsd16',"icsd24", 'pymatgen_sp' and 'wiki' for the  2014 SMACT default, 2016 ICSD, 2024 ICSD, pymatgen structure predictor and Wikipedia (https://en.wikipedia.org/wiki/Template:List_of_oxidation_states_of_the_elements) oxidation states respectively. A filepath to an oxidation states text file can also be supplied as well.

    Returns:
    -------
        allowed_comps (list): Allowed compositions for that chemical system
        in the form [(elements), (oxidation states), (ratios)] if species_unique=True
        or in the form [(elements), (ratios)] if species_unique=False.

    Example usage:
        >>> from smact.screening import smact_filter
        >>> from smact import Element
        >>> els = (Element("Cs"), Element("Pb"), Element("I"))
        >>> comps = smact_filter(els, threshold=5)
        >>> for comp in comps:
        >>>     print(comp)
        [('Cs', 'Pb', 'I'), (1, -4, -1), (5, 1, 1)]
        [('Cs', 'Pb', 'I'), (1, 2, -1), (1, 1, 3)]
        [('Cs', 'Pb', 'I'), (1, 2, -1), (1, 2, 5)]
        [('Cs', 'Pb', 'I'), (1, 2, -1), (2, 1, 4)]
        [('Cs', 'Pb', 'I'), (1, 2, -1), (3, 1, 5)]
        [('Cs', 'Pb', 'I'), (1, 4, -1), (1, 1, 5)]

    Example (using stoichs):
        >>> from smact.screening import smact_filter
        >>> from smact import Element
        >>> comps = smact_filter(els, stoichs=[[1], [1], [3]])
        >>> for comp in comps:
        >>>     print(comp)
        [('Cs', 'Pb', 'I'), (1, 2, -1), (1, 1, 3)]


    """
    compositions = []

    # Get symbols and electronegativities
    symbols = tuple(e.symbol for e in els)
    electronegs = tuple(e.pauling_eneg for e in els)

    if os.path.exists(oxidation_states_set):
        ox_combos = [oxi_custom(e.symbol, oxidation_states_set) for e in els]
    else:
        raise (
            Exception(
                f'{oxidation_states_set} is not valid. Enter either "smact14", "icsd", "pymatgen","wiki" or a filepath to a textfile of oxidation states.'
            )
        )

    for ox_states in itertools.product(*ox_combos):
        # Test for charge balance
        cn_e, cn_r = neutral_ratios(ox_states, stoichs=stoichs, threshold=threshold)
        # Electronegativity test
        if cn_e:
            electroneg_OK = pauling_test(ox_states, electronegs)
            if electroneg_OK:
                compositions.append((symbols, ox_states, ratio))

    # Return list depending on whether we are interested in unique species combinations
    # or just unique element combinations.
    if species_unique:
        return compositions
    else:
        compositions = [(i[0], i[2]) for i in compositions]
        return list(set(compositions))

# suggested new code

In [4]:
def _get_data_rows_custom_ox_states(filename):
    """Generator for datafile entries by row for custom oxidation states lists. Skips rows with no oxidation states for performance."""
    with open(filename) as file:
        for line in file:
            line = line.strip()
            if line[0] != "#" and any(char.isdigit() for char in line):
                yield line.split()


print(list(_get_data_rows_custom_ox_states(oxidation_states_set)))

[['H', '-1', '1'], ['Li', '1'], ['Be', '2'], ['B', '-5', '-3', '-2', '-1', '1', '2', '3'], ['C', '-4', '-3', '-2', '-1', '1', '2', '3', '4'], ['N', '-5', '-4', '-3', '-2', '-1', '1', '2', '3', '4', '5'], ['O', '-2', '-1', '2'], ['F', '-1'], ['Na', '1'], ['Mg', '2'], ['Al', '-3', '3'], ['Si', '-4', '-3', '-2', '-1', '1', '2', '3', '4'], ['P', '-4', '-3', '-2', '-1', '1', '2', '3', '4', '5'], ['S', '-2', '-1', '1', '2', '3', '4', '5', '6'], ['Cl', '-1', '1', '3', '4', '5', '7'], ['K', '1'], ['Ca', '1', '2'], ['Sc', '1', '2', '3'], ['Ti', '1', '2', '3', '4'], ['V', '1', '2', '3', '4', '5'], ['Cr', '1', '2', '3', '4', '5', '6'], ['Mn', '-1', '1', '2', '3', '4', '5', '6', '7'], ['Fe', '-4', '1', '2', '3', '4', '5', '6'], ['Co', '-1', '1', '2', '3', '4', '5'], ['Ni', '1', '2', '3', '4'], ['Cu', '1', '2', '3', '4'], ['Zn', '1', '2', '3', '4'], ['Ga', '-4', '-3', '1', '2', '3', '4'], ['Ge', '-4', '-3', '-2', '-1', '1', '2', '3', '4'], ['As', '-3', '-2', '-1', '1', '2', '3', '4', '5'], ['Se', '

In [17]:
ox_states = {}

for items in _get_data_rows(oxidation_states_set):
    ox_states[items[0]] = [int(oxidationState) for oxidationState in items[1:]]

print(f"ox_states: {ox_states}")

ox_states_custom = {}

for items in _get_data_rows_custom_ox_states(oxidation_states_set):
    ox_states_custom[items[0]] = [int(oxidationState) for oxidationState in items[1:]]

print(f"ox_states_custom: {ox_states_custom}")

ox_states: {'H': [1], 'He': [], 'Li': [1], 'Be': [2], 'B': [3], 'C': [], 'N': [-3], 'O': [-2], 'F': [-1], 'Ne': [], 'Na': [1], 'Mg': [2], 'Al': [3], 'Si': [4], 'P': [5], 'S': [-2], 'Cl': [-1], 'Ar': [], 'K': [1], 'Ca': [2], 'Sc': [3], 'Ti': [4], 'V': [5], 'Cr': [3], 'Mn': [2], 'Fe': [3], 'Co': [2], 'Ni': [2], 'Cu': [2], 'Zn': [2], 'Ga': [3], 'Ge': [4], 'As': [], 'Se': [-2], 'Br': [-1], 'Kr': [2], 'Rb': [1], 'Sr': [2], 'Y': [3], 'Zr': [4], 'Nb': [5], 'Mo': [6], 'Tc': [], 'Ru': [], 'Rh': [3], 'Pd': [2], 'Ag': [1], 'Cd': [2], 'In': [3], 'Sn': [4], 'Sb': [], 'Te': [-2], 'I': [-1], 'Xe': [], 'Cs': [1], 'Ba': [2], 'La': [3], 'Ce': [3], 'Pr': [3], 'Nd': [3], 'Pm': [3], 'Sm': [3], 'Eu': [3], 'Gd': [3], 'Tb': [3], 'Dy': [3], 'Ho': [3], 'Er': [3], 'Tm': [3], 'Yb': [3], 'Lu': [3], 'Hf': [4], 'Ta': [5], 'W': [6], 'Re': [], 'Os': [], 'Ir': [], 'Pt': [4], 'Au': [], 'Hg': [2], 'Tl': [1], 'Pb': [2], 'Bi': [3], 'Po': [4], 'At': [], 'Rn': [], 'Fr': [], 'Ra': [2], 'Ac': [3], 'Th': [4], 'Pa': [5], 'U': [6

In [2]:
lookup_element_oxidation_states_custom(
    "all",
    "/Users/kinga/Documents/my_scripts/SMACT/commonality_oxstates_lists/high_commonality.txt",
    copy=False,
)

Returning the data for all elements.


{'H': [1],
 'Li': [1],
 'Be': [2],
 'B': [3],
 'N': [-3],
 'O': [-2],
 'F': [-1],
 'Na': [1],
 'Mg': [2],
 'Al': [3],
 'Si': [4],
 'P': [5],
 'S': [-2],
 'Cl': [-1],
 'K': [1],
 'Ca': [2],
 'Sc': [3],
 'Ti': [4],
 'V': [5],
 'Cr': [3],
 'Mn': [2],
 'Fe': [3],
 'Co': [2],
 'Ni': [2],
 'Cu': [2],
 'Zn': [2],
 'Ga': [3],
 'Ge': [4],
 'Se': [-2],
 'Br': [-1],
 'Kr': [2],
 'Rb': [1],
 'Sr': [2],
 'Y': [3],
 'Zr': [4],
 'Nb': [5],
 'Mo': [6],
 'Rh': [3],
 'Pd': [2],
 'Ag': [1],
 'Cd': [2],
 'In': [3],
 'Sn': [4],
 'Te': [-2],
 'I': [-1],
 'Cs': [1],
 'Ba': [2],
 'La': [3],
 'Ce': [3],
 'Pr': [3],
 'Nd': [3],
 'Pm': [3],
 'Sm': [3],
 'Eu': [3],
 'Gd': [3],
 'Tb': [3],
 'Dy': [3],
 'Ho': [3],
 'Er': [3],
 'Tm': [3],
 'Yb': [3],
 'Lu': [3],
 'Hf': [4],
 'Ta': [5],
 'W': [6],
 'Pt': [4],
 'Hg': [2],
 'Tl': [1],
 'Pb': [2],
 'Bi': [3],
 'Po': [4],
 'Ra': [2],
 'Ac': [3],
 'Th': [4],
 'Pa': [5],
 'U': [6],
 'Am': [3],
 'Cm': [3],
 'Bk': [3],
 'Cf': [3],
 'Es': [3]}

In [9]:
pprint(
    f"lookup_element_oxidation_states_custom: {lookup_element_oxidation_states_custom('all', oxidation_states_set, copy=False)}"
)

Returning the data for all elements.
("lookup_element_oxidation_states_custom: {'H': [-1, 1], 'Li': [1], 'Be': [2], "
 "'B': [-5, -3, -2, -1, 1, 2, 3], 'C': [-4, -3, -2, -1, 1, 2, 3, 4], 'N': [-5, "
 "-4, -3, -2, -1, 1, 2, 3, 4, 5], 'O': [-2, -1, 2], 'F': [-1], 'Na': [1], "
 "'Mg': [2], 'Al': [-3, 3], 'Si': [-4, -3, -2, -1, 1, 2, 3, 4], 'P': [-4, -3, "
 "-2, -1, 1, 2, 3, 4, 5], 'S': [-2, -1, 1, 2, 3, 4, 5, 6], 'Cl': [-1, 1, 3, 4, "
 "5, 7], 'K': [1], 'Ca': [1, 2], 'Sc': [1, 2, 3], 'Ti': [1, 2, 3, 4], 'V': [1, "
 "2, 3, 4, 5], 'Cr': [1, 2, 3, 4, 5, 6], 'Mn': [-1, 1, 2, 3, 4, 5, 6, 7], "
 "'Fe': [-4, 1, 2, 3, 4, 5, 6], 'Co': [-1, 1, 2, 3, 4, 5], 'Ni': [1, 2, 3, 4], "
 "'Cu': [1, 2, 3, 4], 'Zn': [1, 2, 3, 4], 'Ga': [-4, -3, 1, 2, 3, 4], 'Ge': "
 "[-4, -3, -2, -1, 1, 2, 3, 4], 'As': [-3, -2, -1, 1, 2, 3, 4, 5], 'Se': [-2, "
 "-1, 1, 2, 3, 4, 5, 6], 'Br': [-1, 1, 3, 5, 7], 'Kr': [2], 'Rb': [1], 'Sr': "
 "[2, 4], 'Y': [1, 2, 3, 4], 'Zr': [1, 2, 3, 4], 'Nb': [1, 2, 3, 4, 5, 6], "
 "'Mo': [-1,

In [ ]:
ox_states_custom = {}

for items in _get_data_rows_custom_ox_states_list(oxidation_states_set):
    ox_states_custom[items[0]] = [int(oxidationState) for oxidationState in items[1:]]

elements_pauling = [
    Element(element)
    for element in ordered_elements(1, max_atomic_num)
    if element in ox_states_custom.keys()
    and Element(element).pauling_eneg is not None
    and Element(element).pauling_eneg >= Element("Fr").pauling_eneg
]  # omit elements without Pauling electronegativity (e.g., He, Ne, Ar, ...)

compounds_pauling = list(itertools.combinations(elements_pauling, num_elements))

pool = multiprocessing.Pool(
    processes=(multiprocessing.cpu_count() if num_processes is None else num_processes)
)

results = list(
    tqdm(
        pool.imap_unordered(
            partial(
                smact_filter,
                threshold=max_stoich,
                oxidation_states_set=oxidation_states_set,
            ),
            compounds_pauling,
        ),
        total=len(compounds_pauling),
    )
)
pool.close()
pool.join()

print(results)
smact_allowed = []
for result in results:
    print(result)
    smact_allowed.append(formula_maker(result[0]))
    print(smact_allowed)

  0%|          | 0/4186 [00:01<?, ?it/s]

cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(2, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(2, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is

NameError: name 'ratio' is not defined


cn_r is [(1, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(1, 1)]
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(1, 1)]
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is False
cn_r is []
cn_e is Falsecn_e is False

cn_r is []cn_r is []

cn_e is False
cn_r is []
cn_e is True
cn_e is False
cn_r is [(1, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(1, 1)]
cn_e is True
cn_r is [(2, 1)]
cn_e is True
cn_e is False
cn_r is [(1, 1)]
cn_r is []
cn_e is False
cn_r is []
cn_

cn_e is True
cn_r is [(1, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_e is False
cn_r is []
cn_r is []
cn_e is False
cn_e is Falsecn_r is []

cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is True
cn_r is [(1, 1)]
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is False
cn_r is []
cn_e is Fa

In [ ]:
if os.path.exists(oxidation_states_set):
    ox_combos = [oxi_custom(e.symbol, oxidation_states_set) for e in els]
else:
    raise (Exception(f"{oxidation_states_set} filepath does not exist."))

compositions = []

# Get symbols and electronegativities
symbols = tuple(e.symbol for e in els)
electronegs = [e.pauling_eneg for e in els]


for ox_states in itertools.product(*ox_combos):
    # Test for charge balance
    cn_e, cn_r = neutral_ratios(ox_states, stoichs=stoichs, threshold=threshold)
    # Electronegativity test
    if cn_e:
        electroneg_OK = pauling_test(ox_states, electronegs)
        if electroneg_OK:
            for ratio in cn_r:
                compositions.append((symbols, ox_states, ratio))

# Return list depending on whether we are interested in unique species combinations
# or just unique element combinations.
if species_unique:
    return compositions
else:
    compositions = [(i[0], i[2]) for i in compositions]
    return list(set(compositions))

## _get_data_rows

In [ ]:
from smact.data_loader import _get_data_rows_custom_ox_states_list
from smact.data_loader import _get_data_rows

_el_ox_states_custom = {}

for items in _get_data_rows_custom_ox_states_list(oxidation_states_set):
    _el_ox_states_custom[items[0]] = [
        int(oxidationState) for oxidationState in items[1:]
    ]

elements_valid = list(_el_ox_states_custom.keys())
print(elements_valid)

ox_combos = [
    [_el_ox_states_custom.get(el1.symbol), _el_ox_states_custom.get(el2.symbol)]
    for el1, el2 in compounds_pauling
    if el1 and el2
]


# for key, value in _el_ox_states_custom.items():
#     print(key)
#     print(value)


# all_ox_combos = []
# for els in compounds_pauling:
#     keys_values = []
# if _el_ox_states_custom is not None:
# if copy:
#     # _el_ox_states_custom stores lists -> if copy is set, make an implicit
#     # deep copy.  The elements of the lists are integers, which are
#     # "value types" in Python.

# return list(_el_ox_states_custom[symbol])
# else:
#     return _el_ox_states_custom[symbol]
# else:
#     if _print_warnings:
#         print(f"WARNING: Oxidation states for element {symbol} not found.")
#     return None

In [ ]:
from smact.data_loader import _get_data_rows_custom_ox_states_list

for row in _get_data_rows_custom_ox_states_list(oxidation_states_set):
    print(row)

## oxi_custom

In [ ]:
from smact.data_loader import (
    lookup_element_oxidation_states_custom as oxi_custom,
)

all_ox_combos = []
for els in compounds_pauling:
    symbols = tuple(e.symbol for e in els)
    print(symbols)
    if os.path.exists(oxidation_states_set):
        list_nocopy = [
            oxi_custom(e.symbol, oxidation_states_set, copy=True) for e in els
        ]
        print(list_nocopy)

        # ox_combos = [oxi_custom(e.symbol, oxidation_states_set) for e in els]
        # print(ox_combos)

# Generate compositions in SMACT

In [ ]:
elements_non_pauling = [
    Element(element)
    for element in ordered_elements(1, max_atomic_num)
    if Element(element).pauling_eneg is None
]

In [ ]:
print(elements_pauling[86].symbol)
print(f"{elements_pauling[86].pauling_eneg:.8f}")
print(elements_pauling[54].symbol)
print(f"{elements_pauling[54].pauling_eneg:.8f}")
print(elements_non_pauling[0].symbol)
print([element.symbol for element in elements_non_pauling])
print(len(elements_non_pauling))

In [ ]:
elements_pauling_None = [
    element
    for element in ordered_elements(1, max_atomic_num)
    if Element(element).pauling_eneg is None
]
print(f"elements_pauling_None: {elements_pauling_None}")

elements_pauling_0 = [
    element
    for element in ordered_elements(1, max_atomic_num)
    if Element(element).pauling_eneg == 0
]
print(f"elements_pauling_0: {elements_pauling_0}")

elements_pauling_07 = [
    element
    for element in ordered_elements(1, max_atomic_num)
    if Element(element).pauling_eneg is not None
    if Element(element).pauling_eneg < 0.7
]
print(f"elements_pauling_07: {elements_pauling_07}")

elements_pauling_all = [element for element in ordered_elements(1, max_atomic_num) if ]

print(f"elements_pauling_all len: {len(elements_pauling_all)}, {elements_pauling_all}")

In [6]:
quaternaries = generate_composition_with_smact(
    num_elements=2,
    max_stoich=8,
    max_atomic_num=103,
    num_processes=8,
    save_path="./ternaries_and_quaternaries/df_quaternary_low.pkl",
    oxidation_states_set="./commonality_oxstates_lists/high_commonality.txt",
)

NameError: name 'generate_composition_with_smact' is not defined